In [130]:
# Import PySpark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "10g") \
    .appName('Exam') \
    .getOrCreate() 
from pyspark.sql.functions import *

In [131]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [132]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [133]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

In [134]:
%%html
<style>
.output_subarea.output_text.output_stream.output_stdout > pre {
  width:max-content;
}
.p-Widget.jp-RenderedText.jp-OutputArea-output > pre {
  width:max-content;
}
</style>

In [135]:
%%time
df =  spark.read.parquet('/content/drive/MyDrive/Colab Notebooks/PARQUET/tablas_juntasf.parquet')

CPU times: user 3.28 ms, sys: 1.08 ms, total: 4.36 ms
Wall time: 299 ms


In [136]:
#df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns)).show()

In [137]:
df = df.drop('MCC','Errors', 'Apartment')

# Limpieza e Ingeniería

In [138]:
us_states = ["AK","AL", "AR",
             "AZ","CA","CO",
             "CT", "DC","DE",
             "FL","GA","HI",
             "IA","ID","IL",
             "IN", "KS","KY",
             "LA","MA","MD",
             "ME","MI","MN",
             "MO","MS","MT",
             "NC","ND","NE",
             "NH","NJ","NM",
             "NV","NY","OH",
             "OK","OR","PA",
             "RI","SC","SD",
             "TN","TX","UT",
             "VA","VT","WA",
             "WI","WV","WY"]

In [139]:
from pyspark.sql.types import FloatType, DecimalType
df = df.select('Person',
                 'CurrentAge',
                 'RetirementAge',
                 'BirthYear',
                 'BirthMonth',
                 'Gender',
                 'Address',
                 'City',
                 'State',
                 'Zipcode',
                 'Latitude',
                 'Longitude',
                  regexp_replace(col("PerCapitaIncome-Zipcode"),
                                 "[^\d.]", "").cast(FloatType()).alias("PerCapitaIncome_Zipcode"),
                  regexp_replace(col("YearlyIncome-Person"),
                                 "[^\d.]", "").cast(FloatType()).alias("YearlyIncome_Person"),
                  regexp_replace(col("TotalDebt"),
                                 "[^\d.]", "").cast(FloatType()).alias("TotalDebt"),
                 'FICOScore',
                 'NumCreditCards',
                 'id',
                 'CARDINDEX',
                 'CardBrand',
                 'CardType',
                 'CardNumber',
                 'Expires',
                 'CVV',
                 'HasChip',
                 'CardsIssued',
                  regexp_replace(col("CreditLimit"), "[^\d.]", "").cast(FloatType()).alias("CreditLimit"),
                 'AcctOpenDate',
                 'YearPINlastChanged',
                 'CardonDarkWeb',
                 'User',
                 'Card',
                 'Year',
                 'Month',
                 'Day',
                 'Time',
                  regexp_replace(col("Amount"), "[^\d.]", "").cast(FloatType()).alias("Amount"),
                 'Use_Chip',
                 'Merchant_Name',
                 'Merchant_City',
                 when(col("Merchant_State").isNull(),
                      "ONLINE").otherwise(col("Merchant_State")).alias("Merchant_State"),
                 when(col("Zip").isNull(), "ONLINE").otherwise(col("Zip")).alias("Zip"),
                 'Is_Fraud')

In [140]:
df = df.select("*",
               when(col("Merchant_State").isin(us_states),
                    "US").when(col("Merchant_State").isin("ONLINE"),
                               "ONLINE").otherwise("ABROAD").alias("TransType"))

In [141]:
df = df.select("*",
               when(col("TransType") == 'US', col("Amount")).otherwise(0).alias('USAmount'),
               when(col("TransType") == 'ONLINE', col("Amount")).otherwise(0).alias('OnlineAmount'),
               when(col("TransType") == 'ABROAD', col("Amount")).otherwise(0).alias('AbroadAmount'),
               (2020-col("Year")).alias("Recency"),
               when((col("RetirementAge")-col("CurrentAge")) < 0, 0
                   ).otherwise((col("RetirementAge")-col("CurrentAge"))).alias("RemainingYears")
              )

In [142]:
# Quitamos los clientes que no tienen crédito.
df = df.filter((col("CreditLimit") > 0) & (col("CardType") == 'Credit'))

In [143]:
#df.show(5)

# TAD

In [144]:
f = df.groupBy(['Person', 'CurrentAge','FICOScore', 'RetirementAge',
                'YearlyIncome_Person', 'TotalDebt', 'NumCreditCards','Time','Month','User','Zip','Merchant_State'])\
      .agg((sum("Amount").cast(DecimalType(10,2))).alias("TotalAmount"),
           (avg("Amount").cast(DecimalType(10,2))).alias("AvgAmount"),
           (max("Amount").cast(DecimalType(10,2))).alias("MaxAmount"),
           (min("Amount").cast(DecimalType(10,2))).alias("MinAmount"),
           count("id").alias("TotalTransactions"),
           count(when(col('TransType') == 'US', True)).alias('USTrans'),
           count(when(col('TransType') == 'ONLINE', True)).alias('OnlineTrans'),
           count(when(col('TransType') == 'ABROAD', True)).alias('AbroadTrans'),
           (avg("USAmount").cast(DecimalType(10,2))).alias("AvgUSAmount"),
           (avg("OnlineAmount").cast(DecimalType(10,2))).alias("AvgOnlineAmount"),
           (avg("AbroadAmount").cast(DecimalType(10,2))).alias("AvgAbroadAmount"),
           (max("CreditLimit").cast(DecimalType(10,2))).alias("MaxCreditLimit"),
           (min("CreditLimit").cast(DecimalType(10,2))).alias("MinCreditLimit"),
           (avg("CreditLimit").cast(DecimalType(10,2))).alias("AvgCreditLimit"),
           (max("Recency")).alias("Antiquity"),
           (max("RemainingYears")).alias("RemainingYears")
          )

In [145]:
#f.show()

In [146]:
#f.count()

# Clientes repetidos

In [147]:
f.groupBy("Person").count().orderBy(col("count").desc()).show()

+----------------+-----+
|          Person|count|
+----------------+-----+
|     Simeon Cruz|22015|
|Beckett Gonzalez|17505|
| Addison Faraday|16538|
|      Chris Hill|16187|
| Journey Ramirez|16125|
|    Evelynn Diaz|15741|
|      Corey King|15476|
|       Alisa Abe|15343|
|   Darius Taylor|14717|
|   Gianni Merkel|13977|
|    Rhys Faraday|13945|
|     Bill Merkel|13820|
|   Jakob Roberts|13800|
|  Garrett Macron|13538|
|     Maya Torres|13321|
| Kynlee Martinez|13165|
|    Terry Nguyen|13072|
|   Justice Davis|13011|
|   Heaven Wright|12883|
|      Ryan Anwar|12848|
+----------------+-----+
only showing top 20 rows



In [148]:
f.filter(col("Person") == "Magdalena Farhad").show()

+----------------+----------+---------+-------------+-------------------+---------+--------------+-----+-----+----+------+--------------+-----------+---------+---------+---------+-----------------+-------+-----------+-----------+-----------+---------------+---------------+--------------+--------------+--------------+---------+--------------+
|          Person|CurrentAge|FICOScore|RetirementAge|YearlyIncome_Person|TotalDebt|NumCreditCards| Time|Month|User|   Zip|Merchant_State|TotalAmount|AvgAmount|MaxAmount|MinAmount|TotalTransactions|USTrans|OnlineTrans|AbroadTrans|AvgUSAmount|AvgOnlineAmount|AvgAbroadAmount|MaxCreditLimit|MinCreditLimit|AvgCreditLimit|Antiquity|RemainingYears|
+----------------+----------+---------+-------------+-------------------+---------+--------------+-----+-----+----+------+--------------+-----------+---------+---------+---------+-----------------+-------+-----------+-----------+-----------+---------------+---------------+--------------+--------------+---------

Notemos que tenemos diferente edad y zipcode, por lo tanto nos quedaremos con la edad más actual.

In [149]:
f = f.orderBy(col('CurrentAge').desc()).coalesce(1).dropDuplicates(subset = ['Person'])

In [150]:
#f.count()

In [151]:

pandas_df = f.toPandas()

In [152]:
import numpy as np 

In [153]:
df=pandas_df.copy()

In [170]:
df_nonusa = df[df['Merchant_State'] != 'ONLINE']
print(f"# Transacciones en USA: {len(df_nonusa)}")

# Transacciones en USA: 1160


In [171]:
df_usa = df[df['Merchant_State'] == 'ONLINE']
print(f"# Transacciones fuera USA: {len(df_usa)}")

# Transacciones fuera USA: 136


In [156]:
transaction_count = df_nonusa['Merchant_State'].value_counts().to_frame().reset_index()

In [157]:
transaction_count = transaction_count.rename(columns={"index":"Country","Merchant_State":"Count"})

In [158]:
transaction_count.Country.values

array(['CA', 'TX', 'FL', 'NY', 'OH', 'NC', 'PA', 'IL', 'MI', 'IN', 'NJ',
       'GA', 'VA', 'TN', 'OK', 'MD', 'WA', 'LA', 'MO', 'AZ', 'WI', 'SC',
       'MA', 'AL', 'IA', 'MN', 'MS', 'NV', 'OR', 'CO', 'KY', 'AR', 'CT',
       'KS', 'NM', 'NE', 'HI', 'UT', 'WV', 'NH', 'SD', 'RI', 'ME', 'VT',
       'ID', 'DE', 'MT', 'AK', 'Canada', 'ND', 'DC', 'WY', 'Mexico',
       'Jamaica', 'Japan'], dtype=object)

In [159]:
code_list = []

for country in transaction_count.Country.values:
    small_country = country.lower()
    try:
        response = requests.get("https://restcountries.eu/rest/v2/name/" + small_country + "?fullText=true")
        code_list.append(response.json()[0]['alpha3Code'])
    except:
        code_list.append("")

In [160]:
transaction_count["ISO_Code"] = code_list

In [161]:
transaction_count[transaction_count["ISO_Code"] == ""]

,Country,Count,ISO_Code
0,CA,140,
1,TX,94,
2,FL,82,
3,NY,66,
4,OH,49,
5,NC,48,
6,PA,46,
7,IL,44,
8,MI,37,
9,IN,34,


In [162]:
transaction_count.at[3, "ISO_Code"] = "GBR"
transaction_count.at[13, "ISO_Code"] = "KOR"
transaction_count.at[17, "ISO_Code"] = "BHS"
transaction_count.at[47, "ISO_Code"] = "RUS"
transaction_count.at[54, "ISO_Code"] = "VAT"
transaction_count.at[55, "ISO_Code"] = "MKD"
transaction_count.at[57, "ISO_Code"] = "VNM"
transaction_count.at[91, "ISO_Code"] = "VEN"
transaction_count.at[97, "ISO_Code"] = "MDA"
transaction_count.at[99, "ISO_Code"] = "SYR"
transaction_count.at[107, "ISO_Code"] = "BUR"
transaction_count.at[110, "ISO_Code"] = "IRN"
transaction_count.at[116, "ISO_Code"] = "FSM"
transaction_count.at[123, "ISO_Code"] = "KOS"
transaction_count.at[124, "ISO_Code"] = "TLS"
transaction_count.at[136, "ISO_Code"] = "TZA"
transaction_count.at[140, "ISO_Code"] = "BRN"
transaction_count.at[148, "ISO_Code"] = "COG"
transaction_count.at[166, "ISO_Code"] = "COD"

In [163]:
transaction_count['Count'] = np.log10(transaction_count['Count'].replace(0, np.nan))

In [164]:
fig = px.choropleth(transaction_count, locations="ISO_Code",
                    color="Count",
                    hover_name="Country",
                    color_continuous_scale=px.colors.sequential.Blues,
                    title ="Transacciones alrededor del mundo")
fig.show()

In [165]:
df['Hour'] = df['Time'].apply(lambda value: value.split(":")[0])
df['Minutes'] = df['Time'].apply(lambda value: value.split(":")[1])

In [166]:
convert_dict = {
                'Minutes': np.uint8,
                'Hour': np.uint8,
                'Month': np.uint8,
               }

In [167]:
df = df.astype(convert_dict)

In [168]:
fig = px.histogram(df, x="Month")
fig.update_layout(bargap=0.2, title="Distribución de Transacciones a lo largo de los meses")
fig.show()

In [169]:
fig_hour = px.histogram(df, x="CurrentAge")
fig.update_layout(bargap=0.09, title="Distribución de transacciones por edades")
fig_hour.show()